In [15]:
import pandas as pd
import numpy as np
import os
target_directory = "Z:\\FNS_GA_v5_100_gen_CNN_hyperparam_work\\models\\keff\\data"
pre_process_data = ["keff_training_data.csv", "keff_validation_data.csv"]
void_value = 1.
variable_cassette_2a = True

In [16]:
os.chdir(target_directory)

In [17]:
#def encode(data):
#    print('Shape of data (BEFORE encode): %s' % str(data.shape))
#    encoded = to_categorical(data)
#    print('Shape of data (AFTER  encode): %s\n' % str(encoded.shape))
#    return encoded

In [18]:
#def one_hot_encode_fns(material_value, number_of_materials):
#    zeros_ = np.zeros(number_of_materials)
#    material_array = zeros_
#    material_array[int(material_value)-1] = 1
#    print(material_array)
#one_hot_encode_fns(material_value=4, number_of_materials = 4)

In [19]:
def preprocess_variable_cassette_raw_ga_data(data_file, output_file_name="", fixed_plate_count=120):
    with open(data_file, 'r') as data_file_obj:
        data_master_list = []
        for line in data_file_obj:
            list_ = line.split(",")

            # print(len(list_))
            # print(list_)
            new_list = []
            for value in list_:
                if value.strip() == "":
                    continue
                new_list.append(value.strip())
            # print(len(new_list), len(new_list) - fixed_plate_count - 1)
            # print(new_list)
            size_of_all_patterns = len(new_list) - 1

            size_of_variable_cassette = size_of_all_patterns - fixed_plate_count

            new_list_with_variable_cassette = []
            for count, value in enumerate(new_list):
                new_list_with_variable_cassette.append(value)
                if count == 120:
                    for _ in range(30 - size_of_variable_cassette):
                        new_list_with_variable_cassette.append('1')
            # print("final list", len(new_list_with_variable_cassette), new_list_with_variable_cassette)
            data_master_list.append(new_list_with_variable_cassette)
        # print(len(data_master_list))
        with open(output_file_name, 'w') as output_file_obj:
            for _ in data_master_list:
                write_string = ""
                for __ in _:
                    write_string +=  str(__) + ","
                output_file_obj.write(write_string[:-1] + "\n")

In [20]:
for preproc_ in pre_process_data:
    preprocess_variable_cassette_raw_ga_data(data_file = preproc_, output_file_name = "processed_" + preproc_)

In [21]:
def return_pattern(pat_a, pat_b, pat_c, plate_number):
    row_1 = np.array([pat_c[plate_number], pat_b[plate_number],
                     pat_b[plate_number], pat_a[plate_number]])
    return np.reshape(row_1, (2,2))


In [22]:
#data_ = np.loadtxt("rep_validation_gen_46.csv", delimiter=",")
#X = data_[:,0:150]
#Y = data_[:,150:151]
#zones = [1, 2, 3]
#number_of_plates = 60

def build_3x3_fns_data(data_file_string, zones = [1, 2, 3], number_of_plates = 60, number_of_materials = 4):
    data_ = np.loadtxt(data_file_string, delimiter=",")
    X = data_[:,1:151]
    Y = data_[:,0:1]
    print(X[0])
    print(Y[0])
    pattern_shapes_list = []
    for fns_pattern_count, pattern in enumerate(X): 

        ### Setting patterns for this dataset
        if variable_cassette_2a == False:
            pattern_1a = pattern[0:10]
            pattern_1b = pattern[10:30]
            pattern_1c = pattern[30:50]
            pattern_2a = pattern[50:70]
            pattern_2b = pattern[70:90]
            pattern_2c = pattern[90:110]
            pattern_3a = np.array([void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value])
            pattern_3b = pattern[110:130]
            pattern_3c = pattern[130:150]

            np_four_10_array = np.array([void_value, void_value, void_value, void_value, void_value,
                                         void_value, void_value, void_value, void_value, void_value])
            pattern_1a = np.concatenate((np_four_10_array, pattern_1a))
        
        else:
            pattern_1b = pattern[0:20]
            pattern_1c = pattern[20:40]
            pattern_2b = pattern[40:60]
            pattern_2c = pattern[60:80]

            pattern_3b = pattern[80:100]
            pattern_3c = pattern[100:120]

            np_four_10_array = np.array([void_value, void_value, void_value, void_value, void_value,
                                         void_value, void_value, void_value, void_value, void_value])
            pattern_1a = pattern[120:130]
            pattern_1a = np.concatenate((np_four_10_array, pattern_1a))
            pattern_2a = pattern[130:150]
            pattern_3a = np.array([void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value,
                                   void_value, void_value, void_value, void_value, void_value])

        for plate in range(number_of_plates):
            if plate == 0:
                pattern_array = return_pattern(pattern_1a, pattern_1b, pattern_1c, plate)
                continue

            if plate < 20:
                pattern_ = return_pattern(pattern_1a, pattern_1b, pattern_1c, plate)
                pattern_array = np.concatenate((pattern_array, pattern_))
            if plate < 40 and plate >= 20:
                pattern_ = return_pattern(pattern_2a, pattern_2b, pattern_2c, plate-20)
                pattern_array = np.concatenate((pattern_array, pattern_))
            if plate >= 40:
                pattern_ = return_pattern(pattern_3a, pattern_3b, pattern_3c, plate-40)
                pattern_array = np.concatenate((pattern_array, pattern_))

        pattern_array_reshaped =  np.reshape(pattern_array, (60, 2, 2))
        #print(pattern_array_reshaped.shape)
        ### One hot encoding the 3x3x60 array into a 3x3x60x5 array. Stackoverflow magic:
        ### https://stackoverflow.com/questions/36960320/convert-a-2d-matrix-to-a-3d-one-hot-matrix-numpy
        ### 4 types of materials is hard coded inside np.arange(#)
        pattern_array_reshaped_1he = (np.arange(number_of_materials) == pattern_array_reshaped[...,None]-1).astype(int)

        pattern_shapes_list.append(np.array(pattern_array_reshaped_1he))
        #pattern_array_reshaped_1he_array = np.append(pattern_array_reshaped_1he_array, pattern_array_reshaped_1he)
        print(fns_pattern_count, pattern_array_reshaped_1he.shape)
    pattern_shapes_array = np.array(pattern_shapes_list)
    return pattern_shapes_array, Y

In [23]:
for data in pre_process_data:
    
    pattern_shapes_array, Y = build_3x3_fns_data("processed_"+data)
    np.save("X_"+data, pattern_shapes_array)
    np.save("Y_"+data, Y)

[3. 2. 3. 3. 3. 4. 2. 2. 4. 3. 4. 4. 3. 2. 3. 4. 2. 3. 4. 4. 4. 3. 4. 3.
 4. 4. 3. 2. 4. 3. 4. 4. 3. 3. 4. 2. 3. 3. 4. 4. 4. 2. 3. 3. 3. 3. 3. 4.
 2. 3. 3. 4. 2. 4. 3. 3. 4. 2. 3. 3. 4. 3. 3. 4. 3. 4. 4. 4. 4. 3. 4. 3.
 4. 3. 3. 3. 2. 4. 4. 4. 2. 3. 4. 2. 2. 4. 3. 4. 2. 2. 3. 4. 3. 4. 2. 3.
 2. 2. 4. 4. 4. 2. 3. 2. 4. 3. 2. 3. 2. 2. 4. 3. 4. 3. 3. 3. 4. 4. 4. 3.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 4. 4. 4. 4. 4.
 4. 4. 3. 4. 3. 4.]
[1.00706]
0 (60, 2, 2, 4)
1 (60, 2, 2, 4)
2 (60, 2, 2, 4)
3 (60, 2, 2, 4)
4 (60, 2, 2, 4)
5 (60, 2, 2, 4)
6 (60, 2, 2, 4)
7 (60, 2, 2, 4)
8 (60, 2, 2, 4)
9 (60, 2, 2, 4)
10 (60, 2, 2, 4)
11 (60, 2, 2, 4)
12 (60, 2, 2, 4)
13 (60, 2, 2, 4)
14 (60, 2, 2, 4)
15 (60, 2, 2, 4)
16 (60, 2, 2, 4)
17 (60, 2, 2, 4)
18 (60, 2, 2, 4)
19 (60, 2, 2, 4)
20 (60, 2, 2, 4)
21 (60, 2, 2, 4)
22 (60, 2, 2, 4)
23 (60, 2, 2, 4)
24 (60, 2, 2, 4)
25 (60, 2, 2, 4)
26 (60, 2, 2, 4)
27 (60, 2, 2, 4)
28 (60, 2, 2, 4)
29 (60, 2, 2, 4)
30 (60, 2, 2, 4)
31 (60, 2, 2, 4

529 (60, 2, 2, 4)
530 (60, 2, 2, 4)
531 (60, 2, 2, 4)
532 (60, 2, 2, 4)
533 (60, 2, 2, 4)
534 (60, 2, 2, 4)
535 (60, 2, 2, 4)
536 (60, 2, 2, 4)
537 (60, 2, 2, 4)
538 (60, 2, 2, 4)
539 (60, 2, 2, 4)
540 (60, 2, 2, 4)
541 (60, 2, 2, 4)
542 (60, 2, 2, 4)
543 (60, 2, 2, 4)
544 (60, 2, 2, 4)
545 (60, 2, 2, 4)
546 (60, 2, 2, 4)
547 (60, 2, 2, 4)
548 (60, 2, 2, 4)
549 (60, 2, 2, 4)
550 (60, 2, 2, 4)
551 (60, 2, 2, 4)
552 (60, 2, 2, 4)
553 (60, 2, 2, 4)
554 (60, 2, 2, 4)
555 (60, 2, 2, 4)
556 (60, 2, 2, 4)
557 (60, 2, 2, 4)
558 (60, 2, 2, 4)
559 (60, 2, 2, 4)
560 (60, 2, 2, 4)
561 (60, 2, 2, 4)
562 (60, 2, 2, 4)
563 (60, 2, 2, 4)
564 (60, 2, 2, 4)
565 (60, 2, 2, 4)
566 (60, 2, 2, 4)
567 (60, 2, 2, 4)
568 (60, 2, 2, 4)
569 (60, 2, 2, 4)
570 (60, 2, 2, 4)
571 (60, 2, 2, 4)
572 (60, 2, 2, 4)
573 (60, 2, 2, 4)
574 (60, 2, 2, 4)
575 (60, 2, 2, 4)
576 (60, 2, 2, 4)
577 (60, 2, 2, 4)
578 (60, 2, 2, 4)
579 (60, 2, 2, 4)
580 (60, 2, 2, 4)
581 (60, 2, 2, 4)
582 (60, 2, 2, 4)
583 (60, 2, 2, 4)
584 (60, 2

985 (60, 2, 2, 4)
986 (60, 2, 2, 4)
987 (60, 2, 2, 4)
988 (60, 2, 2, 4)
989 (60, 2, 2, 4)
990 (60, 2, 2, 4)
991 (60, 2, 2, 4)
992 (60, 2, 2, 4)
993 (60, 2, 2, 4)
994 (60, 2, 2, 4)
995 (60, 2, 2, 4)
996 (60, 2, 2, 4)
997 (60, 2, 2, 4)
998 (60, 2, 2, 4)
999 (60, 2, 2, 4)
1000 (60, 2, 2, 4)
1001 (60, 2, 2, 4)
1002 (60, 2, 2, 4)
1003 (60, 2, 2, 4)
1004 (60, 2, 2, 4)
1005 (60, 2, 2, 4)
1006 (60, 2, 2, 4)
1007 (60, 2, 2, 4)
1008 (60, 2, 2, 4)
1009 (60, 2, 2, 4)
1010 (60, 2, 2, 4)
1011 (60, 2, 2, 4)
1012 (60, 2, 2, 4)
1013 (60, 2, 2, 4)
1014 (60, 2, 2, 4)
1015 (60, 2, 2, 4)
1016 (60, 2, 2, 4)
1017 (60, 2, 2, 4)
1018 (60, 2, 2, 4)
1019 (60, 2, 2, 4)
1020 (60, 2, 2, 4)
1021 (60, 2, 2, 4)
1022 (60, 2, 2, 4)
1023 (60, 2, 2, 4)
1024 (60, 2, 2, 4)
1025 (60, 2, 2, 4)
1026 (60, 2, 2, 4)
1027 (60, 2, 2, 4)
1028 (60, 2, 2, 4)
1029 (60, 2, 2, 4)
1030 (60, 2, 2, 4)
1031 (60, 2, 2, 4)
1032 (60, 2, 2, 4)
1033 (60, 2, 2, 4)
1034 (60, 2, 2, 4)
1035 (60, 2, 2, 4)
1036 (60, 2, 2, 4)
1037 (60, 2, 2, 4)
1038 (60

1423 (60, 2, 2, 4)
1424 (60, 2, 2, 4)
1425 (60, 2, 2, 4)
1426 (60, 2, 2, 4)
1427 (60, 2, 2, 4)
1428 (60, 2, 2, 4)
1429 (60, 2, 2, 4)
1430 (60, 2, 2, 4)
1431 (60, 2, 2, 4)
1432 (60, 2, 2, 4)
1433 (60, 2, 2, 4)
1434 (60, 2, 2, 4)
1435 (60, 2, 2, 4)
1436 (60, 2, 2, 4)
1437 (60, 2, 2, 4)
1438 (60, 2, 2, 4)
1439 (60, 2, 2, 4)
1440 (60, 2, 2, 4)
1441 (60, 2, 2, 4)
1442 (60, 2, 2, 4)
1443 (60, 2, 2, 4)
1444 (60, 2, 2, 4)
1445 (60, 2, 2, 4)
1446 (60, 2, 2, 4)
1447 (60, 2, 2, 4)
1448 (60, 2, 2, 4)
1449 (60, 2, 2, 4)
1450 (60, 2, 2, 4)
1451 (60, 2, 2, 4)
1452 (60, 2, 2, 4)
1453 (60, 2, 2, 4)
1454 (60, 2, 2, 4)
1455 (60, 2, 2, 4)
1456 (60, 2, 2, 4)
1457 (60, 2, 2, 4)
1458 (60, 2, 2, 4)
1459 (60, 2, 2, 4)
1460 (60, 2, 2, 4)
1461 (60, 2, 2, 4)
1462 (60, 2, 2, 4)
1463 (60, 2, 2, 4)
1464 (60, 2, 2, 4)
1465 (60, 2, 2, 4)
1466 (60, 2, 2, 4)
1467 (60, 2, 2, 4)
1468 (60, 2, 2, 4)
1469 (60, 2, 2, 4)
1470 (60, 2, 2, 4)
1471 (60, 2, 2, 4)
1472 (60, 2, 2, 4)
1473 (60, 2, 2, 4)
1474 (60, 2, 2, 4)
1475 (60, 2,

1922 (60, 2, 2, 4)
1923 (60, 2, 2, 4)
1924 (60, 2, 2, 4)
1925 (60, 2, 2, 4)
1926 (60, 2, 2, 4)
1927 (60, 2, 2, 4)
1928 (60, 2, 2, 4)
1929 (60, 2, 2, 4)
1930 (60, 2, 2, 4)
1931 (60, 2, 2, 4)
1932 (60, 2, 2, 4)
1933 (60, 2, 2, 4)
1934 (60, 2, 2, 4)
1935 (60, 2, 2, 4)
1936 (60, 2, 2, 4)
1937 (60, 2, 2, 4)
1938 (60, 2, 2, 4)
1939 (60, 2, 2, 4)
1940 (60, 2, 2, 4)
1941 (60, 2, 2, 4)
1942 (60, 2, 2, 4)
1943 (60, 2, 2, 4)
1944 (60, 2, 2, 4)
1945 (60, 2, 2, 4)
1946 (60, 2, 2, 4)
1947 (60, 2, 2, 4)
1948 (60, 2, 2, 4)
1949 (60, 2, 2, 4)
1950 (60, 2, 2, 4)
1951 (60, 2, 2, 4)
1952 (60, 2, 2, 4)
1953 (60, 2, 2, 4)
1954 (60, 2, 2, 4)
1955 (60, 2, 2, 4)
1956 (60, 2, 2, 4)
1957 (60, 2, 2, 4)
1958 (60, 2, 2, 4)
1959 (60, 2, 2, 4)
1960 (60, 2, 2, 4)
1961 (60, 2, 2, 4)
1962 (60, 2, 2, 4)
1963 (60, 2, 2, 4)
1964 (60, 2, 2, 4)
1965 (60, 2, 2, 4)
1966 (60, 2, 2, 4)
1967 (60, 2, 2, 4)
1968 (60, 2, 2, 4)
1969 (60, 2, 2, 4)
1970 (60, 2, 2, 4)
1971 (60, 2, 2, 4)
1972 (60, 2, 2, 4)
1973 (60, 2, 2, 4)
1974 (60, 2,

...

6418 (60, 2, 2, 4)
6419 (60, 2, 2, 4)
6420 (60, 2, 2, 4)
6421 (60, 2, 2, 4)
6422 (60, 2, 2, 4)
6423 (60, 2, 2, 4)
6424 (60, 2, 2, 4)
6425 (60, 2, 2, 4)
6426 (60, 2, 2, 4)
6427 (60, 2, 2, 4)
6428 (60, 2, 2, 4)
6429 (60, 2, 2, 4)
6430 (60, 2, 2, 4)
6431 (60, 2, 2, 4)
6432 (60, 2, 2, 4)
6433 (60, 2, 2, 4)
6434 (60, 2, 2, 4)
6435 (60, 2, 2, 4)
6436 (60, 2, 2, 4)
6437 (60, 2, 2, 4)
6438 (60, 2, 2, 4)
6439 (60, 2, 2, 4)
6440 (60, 2, 2, 4)
6441 (60, 2, 2, 4)
6442 (60, 2, 2, 4)
6443 (60, 2, 2, 4)
6444 (60, 2, 2, 4)
6445 (60, 2, 2, 4)
6446 (60, 2, 2, 4)
6447 (60, 2, 2, 4)
6448 (60, 2, 2, 4)
6449 (60, 2, 2, 4)
6450 (60, 2, 2, 4)
6451 (60, 2, 2, 4)
6452 (60, 2, 2, 4)
6453 (60, 2, 2, 4)
6454 (60, 2, 2, 4)
6455 (60, 2, 2, 4)
6456 (60, 2, 2, 4)
6457 (60, 2, 2, 4)
6458 (60, 2, 2, 4)
6459 (60, 2, 2, 4)
6460 (60, 2, 2, 4)
6461 (60, 2, 2, 4)
6462 (60, 2, 2, 4)
6463 (60, 2, 2, 4)
6464 (60, 2, 2, 4)
6465 (60, 2, 2, 4)
6466 (60, 2, 2, 4)
6467 (60, 2, 2, 4)
6468 (60, 2, 2, 4)
6469 (60, 2, 2, 4)
6470 (60, 2,

6917 (60, 2, 2, 4)
6918 (60, 2, 2, 4)
6919 (60, 2, 2, 4)
6920 (60, 2, 2, 4)
6921 (60, 2, 2, 4)
6922 (60, 2, 2, 4)
6923 (60, 2, 2, 4)
6924 (60, 2, 2, 4)
6925 (60, 2, 2, 4)
6926 (60, 2, 2, 4)
6927 (60, 2, 2, 4)
6928 (60, 2, 2, 4)
6929 (60, 2, 2, 4)
6930 (60, 2, 2, 4)
6931 (60, 2, 2, 4)
6932 (60, 2, 2, 4)
6933 (60, 2, 2, 4)
6934 (60, 2, 2, 4)
6935 (60, 2, 2, 4)
6936 (60, 2, 2, 4)
6937 (60, 2, 2, 4)
6938 (60, 2, 2, 4)
6939 (60, 2, 2, 4)
6940 (60, 2, 2, 4)
6941 (60, 2, 2, 4)
6942 (60, 2, 2, 4)
6943 (60, 2, 2, 4)
6944 (60, 2, 2, 4)
6945 (60, 2, 2, 4)
6946 (60, 2, 2, 4)
6947 (60, 2, 2, 4)
6948 (60, 2, 2, 4)
6949 (60, 2, 2, 4)
6950 (60, 2, 2, 4)
6951 (60, 2, 2, 4)
6952 (60, 2, 2, 4)
6953 (60, 2, 2, 4)
6954 (60, 2, 2, 4)
6955 (60, 2, 2, 4)
6956 (60, 2, 2, 4)
6957 (60, 2, 2, 4)
6958 (60, 2, 2, 4)
6959 (60, 2, 2, 4)
6960 (60, 2, 2, 4)
6961 (60, 2, 2, 4)
6962 (60, 2, 2, 4)
6963 (60, 2, 2, 4)
6964 (60, 2, 2, 4)
6965 (60, 2, 2, 4)
6966 (60, 2, 2, 4)
6967 (60, 2, 2, 4)
6968 (60, 2, 2, 4)
6969 (60, 2,

7417 (60, 2, 2, 4)
7418 (60, 2, 2, 4)
7419 (60, 2, 2, 4)
7420 (60, 2, 2, 4)
7421 (60, 2, 2, 4)
7422 (60, 2, 2, 4)
7423 (60, 2, 2, 4)
7424 (60, 2, 2, 4)
7425 (60, 2, 2, 4)
7426 (60, 2, 2, 4)
7427 (60, 2, 2, 4)
7428 (60, 2, 2, 4)
7429 (60, 2, 2, 4)
7430 (60, 2, 2, 4)
7431 (60, 2, 2, 4)
7432 (60, 2, 2, 4)
7433 (60, 2, 2, 4)
7434 (60, 2, 2, 4)
7435 (60, 2, 2, 4)
7436 (60, 2, 2, 4)
7437 (60, 2, 2, 4)
7438 (60, 2, 2, 4)
7439 (60, 2, 2, 4)
7440 (60, 2, 2, 4)
7441 (60, 2, 2, 4)
7442 (60, 2, 2, 4)
7443 (60, 2, 2, 4)
7444 (60, 2, 2, 4)
7445 (60, 2, 2, 4)
7446 (60, 2, 2, 4)
7447 (60, 2, 2, 4)
7448 (60, 2, 2, 4)
7449 (60, 2, 2, 4)
7450 (60, 2, 2, 4)
7451 (60, 2, 2, 4)
7452 (60, 2, 2, 4)
7453 (60, 2, 2, 4)
7454 (60, 2, 2, 4)
7455 (60, 2, 2, 4)
7456 (60, 2, 2, 4)
7457 (60, 2, 2, 4)
7458 (60, 2, 2, 4)
7459 (60, 2, 2, 4)
7460 (60, 2, 2, 4)
7461 (60, 2, 2, 4)
7462 (60, 2, 2, 4)
7463 (60, 2, 2, 4)
7464 (60, 2, 2, 4)
7465 (60, 2, 2, 4)
7466 (60, 2, 2, 4)
7467 (60, 2, 2, 4)
7468 (60, 2, 2, 4)
7469 (60, 2,

296 (60, 2, 2, 4)
297 (60, 2, 2, 4)
298 (60, 2, 2, 4)
299 (60, 2, 2, 4)
300 (60, 2, 2, 4)
301 (60, 2, 2, 4)
302 (60, 2, 2, 4)
303 (60, 2, 2, 4)
304 (60, 2, 2, 4)
305 (60, 2, 2, 4)
306 (60, 2, 2, 4)
307 (60, 2, 2, 4)
308 (60, 2, 2, 4)
309 (60, 2, 2, 4)
310 (60, 2, 2, 4)
311 (60, 2, 2, 4)
312 (60, 2, 2, 4)
313 (60, 2, 2, 4)
314 (60, 2, 2, 4)
315 (60, 2, 2, 4)
316 (60, 2, 2, 4)
317 (60, 2, 2, 4)
318 (60, 2, 2, 4)
319 (60, 2, 2, 4)
320 (60, 2, 2, 4)
321 (60, 2, 2, 4)
322 (60, 2, 2, 4)
323 (60, 2, 2, 4)
324 (60, 2, 2, 4)
325 (60, 2, 2, 4)
326 (60, 2, 2, 4)
327 (60, 2, 2, 4)
328 (60, 2, 2, 4)
329 (60, 2, 2, 4)
330 (60, 2, 2, 4)
331 (60, 2, 2, 4)
332 (60, 2, 2, 4)
333 (60, 2, 2, 4)
334 (60, 2, 2, 4)
335 (60, 2, 2, 4)
336 (60, 2, 2, 4)
337 (60, 2, 2, 4)
338 (60, 2, 2, 4)
339 (60, 2, 2, 4)
340 (60, 2, 2, 4)
341 (60, 2, 2, 4)
342 (60, 2, 2, 4)
343 (60, 2, 2, 4)
344 (60, 2, 2, 4)
345 (60, 2, 2, 4)
346 (60, 2, 2, 4)
347 (60, 2, 2, 4)
348 (60, 2, 2, 4)
349 (60, 2, 2, 4)
350 (60, 2, 2, 4)
351 (60, 2

771 (60, 2, 2, 4)
772 (60, 2, 2, 4)
773 (60, 2, 2, 4)
774 (60, 2, 2, 4)
775 (60, 2, 2, 4)
776 (60, 2, 2, 4)
777 (60, 2, 2, 4)
778 (60, 2, 2, 4)
779 (60, 2, 2, 4)
780 (60, 2, 2, 4)
781 (60, 2, 2, 4)
782 (60, 2, 2, 4)
783 (60, 2, 2, 4)
784 (60, 2, 2, 4)
785 (60, 2, 2, 4)
786 (60, 2, 2, 4)
787 (60, 2, 2, 4)
788 (60, 2, 2, 4)
789 (60, 2, 2, 4)
790 (60, 2, 2, 4)
791 (60, 2, 2, 4)
792 (60, 2, 2, 4)
793 (60, 2, 2, 4)
794 (60, 2, 2, 4)
795 (60, 2, 2, 4)
796 (60, 2, 2, 4)
797 (60, 2, 2, 4)
798 (60, 2, 2, 4)
799 (60, 2, 2, 4)
800 (60, 2, 2, 4)
801 (60, 2, 2, 4)
802 (60, 2, 2, 4)
803 (60, 2, 2, 4)
804 (60, 2, 2, 4)
805 (60, 2, 2, 4)
806 (60, 2, 2, 4)
807 (60, 2, 2, 4)
808 (60, 2, 2, 4)
809 (60, 2, 2, 4)
810 (60, 2, 2, 4)
811 (60, 2, 2, 4)
812 (60, 2, 2, 4)
813 (60, 2, 2, 4)
814 (60, 2, 2, 4)
815 (60, 2, 2, 4)
816 (60, 2, 2, 4)
817 (60, 2, 2, 4)
818 (60, 2, 2, 4)
819 (60, 2, 2, 4)
820 (60, 2, 2, 4)
821 (60, 2, 2, 4)
822 (60, 2, 2, 4)
823 (60, 2, 2, 4)
824 (60, 2, 2, 4)
825 (60, 2, 2, 4)
826 (60, 2